In [1]:
import requests
import pandas as pd
import json
#from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
#from django.db import transaction
#from mlapp.models import Forecast, Correlation, Feature
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
devId = 'sm-0004'

url_dev = f"http://85.14.6.37:16455/api/posts/?date_range=year&not_res=true&dev={devId}"
url_weather = f"http://85.14.6.37:16456/api/weather/?date_range=year&lat=43.2470&long=27.9291"



In [8]:
response = requests.get(url_weather).json()
dfWeather = None
if response:
    dfWeather = pd.DataFrame(response)
    # Convert the 'created_date' column to datetime
    dfWeather['timestamp'] = pd.to_datetime(dfWeather['timestamp'], errors='coerce')
    dfWeather = dfWeather[~dfWeather['timestamp'].duplicated(keep='first')]
    dfWeather.dropna(subset=['timestamp'], inplace=True)
    dfWeather.set_index('timestamp', inplace=True)
    dfWeather = dfWeather.resample('min').interpolate(method='linear')
    dfWeather.reset_index(inplace=True)
    dfWeather['timestamp'] = dfWeather["timestamp"].values.astype('datetime64[m]')

    print(dfWeather)

                 timestamp           id  temperature  clouds  heatindex  \
0      2024-06-26 20:00:00     1.000000    20.700000    4.00  20.700000   
1      2024-06-26 20:01:00     1.016667    20.686667    4.05  20.686667   
2      2024-06-26 20:02:00     1.033333    20.673333    4.10  20.673333   
3      2024-06-26 20:03:00     1.050000    20.660000    4.15  20.660000   
4      2024-06-26 20:04:00     1.066667    20.646667    4.20  20.646667   
...                    ...          ...          ...     ...        ...   
183896 2024-11-01 12:56:00  2723.933333    17.073333    9.00  17.073333   
183897 2024-11-01 12:57:00  2723.950000    17.080000    9.00  17.080000   
183898 2024-11-01 12:58:00  2723.966667    17.086667    9.00  17.086667   
183899 2024-11-01 12:59:00  2723.983333    17.093333    9.00  17.093333   
183900 2024-11-01 13:00:00  2724.000000    17.100000    9.00  17.100000   

              uv      lat     long  
0       6.000000  43.2265  27.9504  
1       5.916667  43.2265

In [10]:
response = requests.get(url_dev).json()
df_sm = None
date_field_name = "created"
if response:
    df_sm = pd.DataFrame(response)
    
    #Convert the 'created_date' column to datetime
    df_sm['timestamp'] = pd.to_datetime(df_sm[date_field_name], errors='coerce')
    
    #Ensure that 'timestamp' column is in datetime64 format
    df_sm['timestamp'] = df_sm["timestamp"].values.astype('datetime64[m]')
    
    df_sm = df_sm[~df_sm['timestamp'].duplicated(keep='first')]

    # Drop rows where 'timestamp' could not be converted
    df_sm.dropna(subset=['timestamp'], inplace=True)

    # 

    # Set the timestamp as the index
    df_sm.set_index('timestamp', inplace=True)
    
    # Resample to minute frequency, filling any gaps
    
    df_sm = df_sm.resample('min').interpolate(method='linear')
    df_sm = df_sm.round(2)
    df_sm['devId'] = devId
    # Reset index to make timestamp a column again
    df_sm.reset_index(inplace=True)        

    columns_to_drop = [date_field_name, 'grid', 'actualCorr', 'actualProviding', 'providingAmount']
    df_sm.drop(columns=columns_to_drop, inplace=True, errors='ignore') 

    print(df_sm)



                 timestamp    devId   value
0      2024-06-21 19:00:00  sm-0004  146.26
1      2024-06-21 19:01:00  sm-0004  145.99
2      2024-06-21 19:02:00  sm-0004  145.72
3      2024-06-21 19:03:00  sm-0004  145.45
4      2024-06-21 19:04:00  sm-0004  145.18
...                    ...      ...     ...
191156 2024-11-01 12:56:00  sm-0004   68.03
191157 2024-11-01 12:57:00  sm-0004   67.90
191158 2024-11-01 12:58:00  sm-0004   67.77
191159 2024-11-01 12:59:00  sm-0004   67.65
191160 2024-11-01 13:00:00  sm-0004   67.52

[191161 rows x 3 columns]


C:\Users\Georgi\AppData\Local\Temp\ipykernel_12936\3804448315.py:25: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_sm = df_sm.resample('min').interpolate(method='linear')


In [11]:

merged_df = None

if dfWeather is not None and not dfWeather.empty and df_sm is not None and not df_sm.empty:
    common_start_timestamp = max(df_sm['timestamp'].min(), dfWeather['timestamp'].min())
    common_end_timestamp = min(df_sm['timestamp'].max(), dfWeather['timestamp'].max())
    # Trim both DataFrames to start from the common start timestamp and end at the common end timestamp
    power_processed_df_trimmed = df_sm[(df_sm['timestamp'] >= common_start_timestamp) & (df_sm['timestamp'] <= common_end_timestamp)].reset_index(drop=True)
    weather_processed_df_trimmed = dfWeather[(dfWeather['timestamp'] >= common_start_timestamp) & (dfWeather['timestamp'] <= common_end_timestamp)].reset_index(drop=True)

    merged_df = pd.merge(power_processed_df_trimmed, weather_processed_df_trimmed, on='timestamp', how='inner') 

    print(merged_df)        


                 timestamp    devId   value           id  temperature  clouds  \
0      2024-06-26 20:00:00  sm-0004  120.36     1.000000    20.700000    4.00   
1      2024-06-26 20:01:00  sm-0004  120.47     1.016667    20.686667    4.05   
2      2024-06-26 20:02:00  sm-0004  120.58     1.033333    20.673333    4.10   
3      2024-06-26 20:03:00  sm-0004  120.69     1.050000    20.660000    4.15   
4      2024-06-26 20:04:00  sm-0004  120.80     1.066667    20.646667    4.20   
...                    ...      ...     ...          ...          ...     ...   
183896 2024-11-01 12:56:00  sm-0004   68.03  2723.933333    17.073333    9.00   
183897 2024-11-01 12:57:00  sm-0004   67.90  2723.950000    17.080000    9.00   
183898 2024-11-01 12:58:00  sm-0004   67.77  2723.966667    17.086667    9.00   
183899 2024-11-01 12:59:00  sm-0004   67.65  2723.983333    17.093333    9.00   
183900 2024-11-01 13:00:00  sm-0004   67.52  2724.000000    17.100000    9.00   

        heatindex        uv

In [ ]:
data = merged_df
if data is not None and not data.empty:
    train_data = TimeSeriesDataFrame.from_data_frame(
        data,
        id_column="devId",
        timestamp_column="timestamp"
    )
    predictor = TimeSeriesPredictor(
    prediction_length=700,
    path="/autogluon",  # Adjust path as needed
    target="value",
    eval_metric="MASE",
    freq='T'  # Specify minute frequency
    )
    predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=50,
    )
    predictions = predictor.predict(train_data)

    for index, row in predictions.iterrows():            
        timestamp = index[1]                
        mean_value = round(row['mean'], 2)            
        print(f"Timestamp:{timestamp} || PredictedValue: {mean_value}")
        
        
